In [0]:
dbutils.secrets.listScopes()

In [0]:
jdbcHostname = "j2dtechnologies.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "j2d_training_db"
user_name = dbutils.secrets.get(scope="j2dtech-secrets3", key="azure-sql-user-name")
user_password = dbutils.secrets.get(scope="j2dtech-secrets3", key="azure-sql-password")

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}"

connection_properties = {
  "user": user_name,
  "password": user_password,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
query = "(select TABLE_SCHEMA, TABLE_NAME from INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE') as temp"

table_names_df = spark.read.jdbc(
    url=jdbcUrl,
    table=query,
    properties=connection_properties
)

table_names_df.display()

In [0]:
table_names_df.collect()

In [0]:
dbutils.fs.mounts()

In [0]:
for row in table_names_df.collect():
    schema = row["TABLE_SCHEMA"]
    table_name = row["TABLE_NAME"]
    full_table_name = f"{schema}.{table_name}"
    print(f"Processing table: {full_table_name}")

    df = spark.read.jdbc(
        url=jdbcUrl,
        table=full_table_name,
        properties=connection_properties
    )

    df.write.format("csv").mode("overwrite").save(f"/mnt/j2dadlscontainer/sql_tables/{table_name}")

In [0]:
customer_table = (
    spark.read
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("dbtable", "Customers")
    .option("user", user_name)
    .option("password", user_password)
    .load()
)

customer_table.display()